## Imports and Parser Instantiation

In [1]:
import scripts.sql_parser.sqlite_parser as sql

NC_PATH = '../../data_sets/db/northwind.db'
SC_PATH = '../../data_sets/db/sakila.db'
CC_PATH = '../../data_sets/db/covid19.db'

# instantiate parser objects based on the db path
nc, sc, cc = sql.SqlParser(NC_PATH), sql.SqlParser(SC_PATH), sql.SqlParser(CC_PATH)

# create a table reference for each db for faster querying
nct, sct, cct = nc.table, sc.table, cc.table

### North Wind Tests

In [2]:
nc.q(query=sql.ALL_TABLES)

,name
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


In [3]:
sct(name='Country', limit=-3)


SELECT *
FROM Country 
ORDER BY 1 DESC
LIMIT 3


,country_id,country,last_update
0,109,Zambia,2021-03-06 15:51:49
1,108,Yugoslavia,2021-03-06 15:51:49
2,107,Yemen,2021-03-06 15:51:49


In [4]:
nct(name='Order Details', limit = 10, distinct=True, starts_with=('unitprice', 4), order_by=['Quantity', 1])


SELECT DISTINCT *
FROM "Order Details" 
WHERE unitprice LIKE "4%"
ORDER BY Quantity ASC
LIMIT 10


,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10462,13,4.8,1,0.0
1,11083,28,45.6,1,0.0
2,11091,63,43.9,1,0.0
3,11107,43,46.0,1,0.0
4,11115,24,4.5,1,0.0
5,11145,28,45.6,1,0.0
6,11149,24,4.5,1,0.0
7,11165,28,45.6,1,0.0
8,11194,43,46.0,1,0.0
9,11207,28,45.6,1,0.0


In [5]:
nct(name='Order Details', cols=['avg=UnitPrice:UP Avg', 'max=UnitPrice:UP Max', 'min=UnitPrice:UP Min', 'count=UnitPrice:UP Count'])


SELECT AVG(UnitPrice) AS "UP Avg",
	MAX(UnitPrice) AS "UP Max",
	MIN(UnitPrice) AS "UP Min",
	COUNT(UnitPrice) AS "UP Count"
FROM "Order Details" 



,UP Avg,UP Max,UP Min,UP Count
0,28.850379,263.5,2,609283


### Sakila Tests

In [6]:
sc.q(query=sql.ALL_TABLES)

,name
0,actor
1,country
2,city
3,address
4,language
5,category
6,customer
7,film
8,film_actor
9,film_category


In [7]:
sct(name='Country', limit=-2)


SELECT *
FROM Country 
ORDER BY 1 DESC
LIMIT 2


,country_id,country,last_update
0,109,Zambia,2021-03-06 15:51:49
1,108,Yugoslavia,2021-03-06 15:51:49


In [8]:
sct(name='film', limit=1)


SELECT *
FROM film 
LIMIT 1


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00


In [9]:
sct(name='film_actor', limit=1)


SELECT *
FROM film_actor 
LIMIT 1


,actor_id,film_id,last_update
0,1,1,2021-03-06 15:52:45


In [10]:
sct(name='film_category', limit=1)


SELECT *
FROM film_category 
LIMIT 1


,film_id,category_id,last_update
0,1,6,2021-03-06 15:53:28


## Experiments


In [11]:
nc.q(query='''
SELECT
    Orders.OrderDate,
    Orders.RequiredDate,
    Customers.CompanyName,
    Customers.ContactName
FROM
    Orders
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
''')

,OrderDate,RequiredDate,CompanyName,ContactName
0,2016-07-04,2016-08-01,Vins et alcools Chevalier,Paul Henriot
1,2016-07-05,2016-08-16,Toms Spezialitäten,Karin Josephs
2,2016-07-08,2016-08-05,Hanari Carnes,Mario Pontes
3,2016-07-08,2016-08-05,Victuailles en stock,Mary Saveley
4,2016-07-09,2016-08-06,Suprêmes délices,Pascale Cartrain
...,...,...,...,...
16277,2012-12-26 04:58:22,2013-01-07 03:47:01,White Clover Markets,Karl Jablonski
16278,2022-08-05 06:33:39,2022-09-01 05:09:08,Wolski Zajazd,Zbyszek Piestrzeniewicz
16279,2022-02-09 08:20:12,2022-02-19 13:46:17,Gourmet Lanchonetes,André Fonseca
16280,2020-04-27 23:05:30,2020-05-22 20:22:49,Blondesddsl père et fils,Frédérique Citeaux


In [12]:
nc.q(query='''
SELECT DISTINCT od.UnitPrice
FROM "Order Details" AS od
WHERE od.UnitPrice >= 43.0
	AND od.UnitPrice <= 70.0
ORDER BY od.UnitPrice DESC
LIMIT 5 OFFSET 2
''')

,UnitPrice
0,55.0
1,53.0
2,50.0
3,49.3
4,46.0


In [13]:
nct(name='Order Details', cols=['UnitPrice'], distinct=True, where=['&20.0 >= UnitPrice <= 70.0'], order_by=['UnitPrice', -1], limit=5, offset=2)


SELECT DISTINCT UnitPrice
FROM "Order Details" 
WHERE UnitPrice BETWEEN 20.0 and 70.0
ORDER BY UnitPrice DESC
LIMIT 5 OFFSET 2



,UnitPrice
0,55.0
1,53.0
2,50.0
3,49.3
4,46.0
